In [1]:
import numpy as np
!pip install gurobipy
import gurobipy as gp
GRB = gp.GRB
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 35.9 MB/s eta 0:00:00


In [12]:
def modelisationBP(s,C):
  m = len(s)

  #Modèle et Variables
  model = gp.Model()
  x = model.addMVar((m,m), vtype=GRB.BINARY)
  y = model.addMVar((m,),  vtype=GRB.BINARY)

  #Contraintes
  for i in range(m):
    model.addConstr(x[i, :].sum() == 1)
  for j in range(m):
    somme = 0
    for i in range(m):
      somme += s[i]*x[i,j]
    model.addConstr(somme <= C*y[j])

  #Ajouter des containtes pour briser la symétrie
  for j in range(m-1):
    model.addConstr(y[j]>=y[j+1])


  #Objectif
  model.setObjective(y.sum())

  #Solveur
  model.optimize()

  return x.X, y.X

In [13]:
s = [6,6,5,5,5,4,4,4,4,2,2,2,2,3,3,7,7,5,5,8,8,4,4,5]
C = 9
m = len(s)
x,y = modelisationBP(s,C)

#Affichage
for j in range(m):
  if y[j]==1:
    print(f"Dans la bin {j}, on place : ")
    for i in range(m):
      if x[i,j]==1:
        print(f"\t Item {i} de longeur {s[i]}")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 71 rows, 600 columns and 1222 nonzeros
Model fingerprint: 0x14674a9e
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 18.0000000
Presolve removed 14 rows and 152 columns
Presolve time: 0.03s
Presolved: 57 rows, 448 columns, 910 nonzeros
Variable types: 0 continuous, 448 integer (448 binary)

Root relaxation: objective 1.244444e+01, 102 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [28]:
import itertools

def constructAllPatterns(s,C):
  m = len(s)

  #Génération de tous les patterns respectant la contrainte <= C
  patternspossible = []
  bornesupp = [range(1+C//s[i]) for i in range(len(s))]

  for pattern in itertools.product(*bornesupp):
    if sum([s[i]*pattern[i] for i in range(m)]) <= C:
      patternspossible.append(pattern)

  #Gardons uniquelent les patterns "non-dominés"
  patternstokeep = []
  for i in range(len(patternspossible)):
    for j in range(len(patternspossible)):
      if i!=j:
        if all(patternspossible[i][k]<=patternspossible[j][k] for k in range(m)):
          break
    else:
      patternstokeep.append(patternspossible[i])

  return np.array(patternstokeep).T

In [30]:
s = np.array([6,5,4,3,7,8,2])
d = np.array([2,6,6,2,2,2,4])
C = 9

A = constructAllPatterns(s,C)
print(A)

P = A.shape[1]
print(A.shape)

#Modèle et Variables
model = gp.Model()
z = model.addMVar((P,), vtype=GRB.INTEGER)

#Contraintes
model.addConstr(A@z >= d)

#Objectif
model.setObjective(z.sum())

#Solveur
model.optimize()

print(z.X)
for i in range(P):
  if z[i].X>0:
    print(f"On produit {z[i].X} fois le pattern {A[:,i]}")


[[0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0]
 [0 0 0 0 0 0 1 1 2 0 0 1 0 0]
 [0 0 0 1 2 3 0 1 0 0 1 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [4 0 1 3 1 0 2 1 0 2 0 0 1 0]]
(7, 14)
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 14 columns and 25 nonzeros
Model fingerprint: 0x53b3b557
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Found heuristic solution: objective 14.0000000
Presolve removed 2 rows and 4 columns
Presolve time: 0.00s
Presolved: 5 rows, 10 columns, 20 nonzeros
Variable types: 0 continuous, 10 integer (0 binary)

Root relaxation: objective 1.250000e+0

In [31]:
s = [18,19]
C = 147
A = constructAllPatterns(s,C)
print(A)
print(A.shape)

[[0 1 2 3 5 6 7 8]
 [7 6 5 4 3 2 1 0]]
(2, 8)


In [63]:
#Branch & Bound Générique

import heapq
import numpy as np

def genericBB(root):

  #Définition du pool de noeuds
  pool = []

  #Ajout de la racine dans le pool de noeuds
  heapq.heappush(pool,root)

  #Pire valeur de l'obj
  #On pourrait faire tourner l'heuristique ici
  OPT = np.inf
  bestsol = None
  nbnodes = 0

  while pool:
    node = heapq.heappop(pool)
    nbnodes += 1

    if node.isLeaf():
      if node.getLowerBound() < OPT:
      #on est content, on a trouvé la meilleure solution
        OPT = node.getLowerBound()
        bestsol = copy.deepcopy(node)

    else:
      for child in node.getChildren():
        if child.getLowerBound() < OPT:
          heapq.heappush(pool, child)


  if bestsol is None:
      raise Exception("Aucune solution trouvée. Vérifie que les feuilles sont atteintes correctement.")

  return bestsol, nbnodes

In [61]:
import copy

class Node:
  s = []
  C = []
  m = -1

  def __init__(self, level, s, C):
    self.level = level
    self.bins = []
    Node.s = s
    Node.C = C
    Node.m = len(s)

  def getLowerBound(self):
    return len(self.bins) #très nul, à améliorer !

  def isLeaf(self):
    if self.level == Node.m:
        print("Feuille atteinte !")
    return self.level == Node.m


  def getChildren(self):
    children = []
    current_item = Node.s[self.level]

    # 1. Ajouter une nouvelle bin avec l'item
    child1 = copy.deepcopy(self)
    child1.level += 1
    child1.bins.append([current_item])
    children.append(child1)

    # 2. Essayer de l'ajouter à chaque bin existante
    for i in range(len(self.bins)):
        if sum(self.bins[i]) + current_item <= Node.C:
            child2 = copy.deepcopy(self)
            child2.level += 1
            # ⚠️ on doit refaire une deepcopy pour modifier en sécurité
            child2.bins[i] = child2.bins[i] + [current_item]
            children.append(child2)

    return children


  def __lt__(self,other):
    return self.getLowerBound() < other.getLowerBound()


In [ ]:
s = [6,6,5,5,5,4,4,4,4,2,2,2,2,3,3,7,7,5,5,8,8,4,4,5]
C = 9
root = Node(0, s, C)

bestsol, nbnodes = genericBB(root)

print(f"Le nombre optimal de bin est : {len(bestsol.bins)}")
print(f"Le nombre de noeuds visités est : {nbnodes}")
for i, b in enumerate(bestsol.bins):
    print(f"Bin {i+1} : {b}")


In [40]:
import copy
import heapq

class Node:
    s = []
    C = 0
    m = 0

    def __init__(self, level, s, C):
        self.level = level
        self.bins = []
        Node.s = s
        Node.C = C
        Node.m = len(s)

    def getLowerBound(self):
        # Borne inférieure : First-Fit-Decreasing
        remaining_items = Node.s[self.level:]
        sorted_items = sorted(remaining_items, reverse=True)
        bins = []
        for item in sorted_items:
            placed = False
            for b in bins:
                if sum(b) + item <= Node.C:
                    b.append(item)
                    placed = True
                    break
            if not placed:
                bins.append([item])
        return len(self.bins) + len(bins)

    def isLeaf(self):
        return self.level == Node.m

    def getChildren(self):
        children = []

        item = Node.s[self.level]

        # Essayer de le mettre dans une bin existante
        for i in range(len(self.bins)):
            if sum(self.bins[i]) + item <= Node.C:
                child = copy.deepcopy(self)
                child.level += 1
                child.bins[i].append(item)
                children.append(child)

        # Sinon créer une nouvelle bin
        child = copy.deepcopy(self)
        child.level += 1
        child.bins.append([item])
        children.append(child)

        return children

    def __lt__(self, other):
        return self.getLowerBound() < other.getLowerBound()


In [41]:
def genericBB(root):
    best_solution = None
    best_obj = float('inf')
    nbnodes = 0

    heap = []
    heapq.heappush(heap, root)

    while heap:
        node = heapq.heappop(heap)
        nbnodes += 1

        lb = node.getLowerBound()
        if lb >= best_obj:
            continue  # Pruning

        if node.isLeaf():
            if len(node.bins) < best_obj:
                best_obj = len(node.bins)
                best_solution = node
        else:
            children = node.getChildren()
            for child in children:
                if child.getLowerBound() < best_obj:
                    heapq.heappush(heap, child)

    return best_solution, nbnodes


In [42]:
s = [6,6,5,5,5,4,4,4,4,2,2,2,2,3,3,7,7,5,5,8,8,4,4,5]
C = 9

root = Node(0, s, C)
bestsol, nbnodes = genericBB(root)

print(f"Le nombre optimal de bins est : {len(bestsol.bins)}")
print(f"Nombre de noeuds visités : {nbnodes}")

for i, b in enumerate(bestsol.bins):
    print(f"Bin {i+1} : {b}")


Le nombre optimal de bins est : 13
Nombre de noeuds visités : 643
Bin 1 : [6, 3]
Bin 2 : [6, 2]
Bin 3 : [5, 4]
Bin 4 : [5, 4]
Bin 5 : [5, 4]
Bin 6 : [4, 2, 2]
Bin 7 : [2, 7]
Bin 8 : [3, 5]
Bin 9 : [7]
Bin 10 : [5, 4]
Bin 11 : [8]
Bin 12 : [8]
Bin 13 : [4, 5]
